In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [4]:
DATA_IN_PATH = './data_in/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA)
reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

In [12]:
vectorizer = TfidfVectorizer(min_df=0.0, sublinear_tf = True,
                             ngram_range=(1,3), max_features=5000, analyzer='char')
x = vectorizer.fit_transform(reviews)
# ngram_range-(1,3) : 단어가 feature 가 되므로 그 범위
# max_feature : 내부적으로 tf 를 구할 떄, 길이에 한정을 주는 것

- min_df : 특정 토큰의 df(특정 단어 t가 등장한 문서의 ㅅ) ㄱ밧이 이 값보다 더 적게 나오면 벡터화 과정에서 제거함
- analyzer
    a. 분석하기 위한 n-gram 기준 단위, 'word'와 'char'2가지 옵션을 제공
    b. 'word'의 경우 단어를 하나 단위로 하는 것, 'char' 는 문자 하나를 단위로 하는 것
- sublinear_tf
    a. 문서의 단어 빈도 수에 대한 smooothing 여부를 설정
    b. analyzer의 단위를 'char'로 했기 때문에 tf값이 너무 큰 값으로 나옴
    c. 이 를 전체적으로 줄이는 스케일링의 일환으로 tf => 1 + log(tf) 으로 대체함
- ngram_range : ngram 에서 사용될 단위 수 n의 범위 

In [14]:
features = vectorizer.get_feature_names()
print(features)
# blank도 여게서는 문자
# 알바벳이 1개짜리, 2개짜리 등등 모두 조합을 하는 것

[' ', ' a', ' aa', ' ab', ' ac', ' ad', ' ae', ' af', ' ag', ' ah', ' ai', ' ak', ' al', ' am', ' an', ' ap', ' ar', ' as', ' at', ' au', ' av', ' aw', ' ax', ' az', ' b', ' b ', ' ba', ' bb', ' be', ' bi', ' bl', ' bo', ' br', ' bu', ' by', ' c', ' c ', ' ca', ' ce', ' cg', ' ch', ' ci', ' cl', ' co', ' cr', ' cu', ' cy', ' d', ' da', ' de', ' di', ' do', ' dr', ' du', ' dv', ' dw', ' dy', ' e', ' e ', ' ea', ' eb', ' ec', ' ed', ' ee', ' ef', ' eg', ' ei', ' el', ' em', ' en', ' ep', ' eq', ' er', ' es', ' et', ' eu', ' ev', ' ex', ' ey', ' f', ' f ', ' fa', ' fb', ' fe', ' fi', ' fl', ' fo', ' fr', ' fu', ' fx', ' g', ' g ', ' ga', ' ge', ' gh', ' gi', ' gl', ' go', ' gr', ' gu', ' gw', ' gy', ' h', ' h ', ' ha', ' hb', ' he', ' hi', ' hm', ' ho', ' hu', ' hy', ' i', ' ia', ' ic', ' id', ' ig', ' ii', ' il', ' im', ' in', ' ir', ' is', ' it', ' iv', ' j', ' j ', ' ja', ' je', ' ji', ' jo', ' jr', ' ju', ' k', ' k ', ' ka', ' ke', ' kh', ' ki', ' kl', ' kn', ' ko', ' kr', ' ku', ' ky

In [15]:
print(x.shape)

(25000, 5000)


In [16]:
RANDOM_SEED = 42
TEST_SPLIT = 0.2

y = np.array(sentiments)
x_train, x_eval, y_train, y_eval = \
train_test_split(x,y, test_size = TEST_SPLIT, random_state=RANDOM_SEED)

In [18]:
lgs = LogisticRegression(class_weight='balanced')
lgs.fit(x_train, y_train)

LogisticRegression(class_weight='balanced')

- class_weight='balanced'  
a. 타깃의 비율이 고르지 않을 경우 예를 들면, 이중 분류 문제에서 학습데이터의 구성이 특정한 한 쪽의 타깃으로 쏠릴 경우 데이터가 적은 타깃은 충분히 학습하지 못함  
b. 이런 언밸런스를 극복하기 위해 데이터가 적은 타깃에 대한 손실함수를 수행할 때, 가중치 벡터 W 에 더 많은 가중치를 부여함  
c. 이 때문에 커진 오차를 줄이기 위해 모델은 데이터가 적은 타깃에 대한 학습에 더 치중하게 됨  
d. 이 때 사용하는 W 에 곱하는 가중치는 다음과 같음  
wj=n_samples / (n_classes * n_samplesj)  
wj 각 클래스에 대한 가중치  
n_samples 전체 샘플의 수  
n_classes 전체 클래스 종류의 수(이진 분류는 2)  
n_samplesj 각 클래스의 샘플 수

In [20]:
predicted = lgs.predict(x_eval)

In [21]:
print(predicted)

[0 1 0 ... 0 0 0]


In [23]:
print('Accuracy: %f' % lgs.score(x_eval, y_eval))

Accuracy: 0.859800


In [24]:
TEST_CLEAN_DATA = 'test_clean.csv'
test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

In [25]:
print(test_data.head())

                                              review          id
0  naturally film main themes mortality nostalgia...  "12311_10"
1  movie disaster within disaster film full great...    "8348_2"
2  movie kids saw tonight child loved one point k...    "5828_4"
3  afraid dark left impression several different ...    "7186_2"
4  accurate depiction small time mob life filmed ...   "12128_7"


In [26]:
testDataVecs = vectorizer.transform(test_data['review'])

In [28]:
test_predicted = lgs.predict(testDataVecs)
print(test_predicted)

[1 0 1 ... 0 1 0]


In [30]:
# 캐글이 제출하기위 해 아이디와 정답이 있는 파일을 별도로 만들어서 저장 
DATA_OUT_PATH = './data_out/'
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

answer_dataset = pd.DataFrame({'id': test_data['id'], 'sentiment': test_predicted})

In [31]:
print(answer_dataset.head())

           id  sentiment
0  "12311_10"          1
1    "8348_2"          0
2    "5828_4"          1
3    "7186_2"          0
4   "12128_7"          1


In [32]:
answer_dataset.to_csv(DATA_OUT_PATH + 'lgs_tfidf_answer.csv', index=False, quoting=3)

완성된 파일을 Late Submission 에 제출